We set our block size.

In [49]:
B_BITS = 128
B_BYTES = B_BITS // 8

ROUNDS = 16

Let's define some very simple "f" functions to facilitate our understanding.

In [50]:
def f_const_0(r, k_i):
    return b"\x00" * (B_BYTES // 2)


def f_const_1(r, k_i):
    return b"\x01" * (B_BYTES // 2)


def f_xor(r, k_i):
    return bytes([x ^ y for x, y in zip(r, k_i)])

We need a simple function to get a round key (`k_i`). In this case, we perform a simple shift operation - shifting a whole byte.

In [51]:
from collections import deque


def k_i(master_key, i, mode):
    r_i = deque(master_key)
    if mode == "encrypt":
        r_i.rotate(i)
    else:
        r_i.rotate(-i-1) # -1 here, because the first round has index "0"
    return bytes(r_i)

In [52]:
from binascii import hexlify


def round(i, l_i, r_i, f_i, k_i):
    substitution = f_i(r_i, k_i)
    r_pi = bytes([r_b ^ r_k_b for r_b, r_k_b in zip(l_i, substitution)])
    print(
        f"{i:2d} " 
        f"l = {hexlify(l_i)}, r = {hexlify(r_i)}, k = {hexlify(k_i)}, f_i(r_i,k_i) = {hexlify(substitution)}, {hexlify(r_pi)}"
    )
    return (r_i, r_pi)

In [53]:
def process_block(m, f, master_k, mode):
    l = m[0 : (B_BYTES // 2)]
    r = m[(B_BYTES // 2) :]
    for i in range(0, ROUNDS):
        (l, r) = round(i, l, r, f, k_i(master_k, i, mode))

    return r + l

For the time beeing we don't care about messages which don't have a  size that is not a multiple of the block size.

In [54]:
def process_message(m_in, f, k, mode):
    assert len(m_in) % B_BYTES == 0

    m_out = b""
    for b in range(0, len(m_in) // B_BYTES):
        m_out += process_block(m_in[b * B_BYTES : (b + 1) * B_BYTES], f, k, mode)
    return m_out

In [55]:
def master_k(key_candidate):
    if (len(key_candidate) <= B_BYTES//2):
        return bytes(key_candidate + "\x00"*(B_BYTES//2-len(key_candidate)),"ascii")
    else:
        return bytes(key_candidate[0:B_BYTES//2],"ascii")

In [56]:
pwd = "A Password!"
# pwd = "A"*B_BYTES
key = master_k(pwd)
m_16x8_0s = b"\x00"*B_BYTES
m_16x8 = b"This is a test.."
m_64x8 = b"This is a very large test..... not so large after all, isn't it?"
C = process_message(m_16x8*64, f_xor, key, "encrypt")
print(f"=> {hexlify(C)}, as string={C}")
E = process_message(C, f_xor, key, "decrypt")
print(f"=> {hexlify(E)}, as string={E}")

 0 l = b'5468697320697320', r = b'6120746573742e2e', k = b'412050617373776f', f_i(r_i,k_i) = b'2000240400075941', b'74684d77206e2a61'
 1 l = b'6120746573742e2e', r = b'74684d77206e2a61', k = b'6f41205061737377', f_i(r_i,k_i) = b'1b296d27411d5916', b'7a09194232697738'
 2 l = b'74684d77206e2a61', r = b'7a09194232697738', k = b'776f412050617373', f_i(r_i,k_i) = b'0d6658626208044b', b'790e151542662e2a'
 3 l = b'7a09194232697738', r = b'790e151542662e2a', k = b'73776f4120506173', f_i(r_i,k_i) = b'0a797a5462364f59', b'70706316505f3861'
 4 l = b'790e151542662e2a', r = b'70706316505f3861', k = b'7373776f41205061', f_i(r_i,k_i) = b'03031479117f6800', b'7a0d016c5319462a'
 5 l = b'70706316505f3861', r = b'7a0d016c5319462a', k = b'617373776f412050', f_i(r_i,k_i) = b'1b7e721b3c58667a', b'6b0e110d6c075e1b'
 6 l = b'7a0d016c5319462a', r = b'6b0e110d6c075e1b', k = b'50617373776f4120', f_i(r_i,k_i) = b'3b6f627e1b681f3b', b'4162631248715911'
 7 l = b'6b0e110d6c075e1b', r = b'4162631248715911', k = b'205

Print some frequency information to see the inadequacy of our cipher/function f/subkey generation algo.

In [57]:
from array import array
freq = array('L',(0 for i in range(0,256)))
for i in range(0,len(C)):
    freq[C[i]] += 1

x_labels = []
x_data = []
for i in range(0,256):
    if(freq[i] > 0):
        x_labels.append(i)
        x_data.append(freq[i])
        print(f"{i:2x}", "=>", freq[i])

 4 => 64
 d => 64
 e => 64
19 => 128
1c => 64
1f => 64
35 => 64
36 => 64
3f => 128
42 => 64
45 => 64
74 => 64
7a => 128
